# Sprint1 機械学習スクラッチ 決定木

## 【問題1】不純度を求める関数
ノード の ジニ不純度 を計算する関数を作成してください。ノード 
$t$に対するジニ不純度 $I(t)$は以下の数式で求まります。クラスが混じり合っているほどジニ不純度は高くなります。
$$
I(t) = 1-\sum_{i=1}^{K}P^2(C_i|t) = 1-\sum_{i=1}^{K}(\frac{N_{t,i}}{N_{t,all}})^{2}
$$
$t$ : ノードのインデックス


$i$ : クラスのインデックス


$K$ : クラスの数


$C_i$ : i番目のクラス


$P(C_{i}|t)$ :　t番目のノードにおける$Ci$の割合


$N_{t,i}$ : t番目のノードのi番目のクラスに属するサンプル数


$N_{t,all}$ : t番目のノードのサンプルの総数

まずは簡単な例を作り、手計算と関数の結果を比較してください。<br>
《例》<br>
・クラス1:サンプル数15, クラス2:サンプル数15 → ジニ不純度0.500<br>
・クラス1:サンプル数15, クラス2:サンプル数15, クラス3:サンプル数15 → ジニ不純度0.667<br>
・クラス1:サンプル数18, クラス2:サンプル数12 → ジニ不純度0.480<br>
・クラス1:サンプル数30, クラス2:サンプル数0 → ジニ不純度0.000<br>

In [7]:
import numpy as np

In [29]:
def test(n1, n2):
    print((n1/(n1+n2))**2)
    print((n2/(n1+n2))**2)
    return 1 - ((n1/(n1+n2))**2 + (n2/(n1+n2))**2)

In [30]:
test(15, 15)
    

0.25
0.25


0.5

In [34]:
X = np.array([[1, 5],
              [2, 5],
              [3, 5],
              [1, 3],
              [2, 4],
              [3, 3],
              [5, 3],
              [4, 2],
              [3, 1],
              [5, 1]])
X.shape

(10, 2)

In [35]:
y = np.array([0, 0, 0, 0, 0, 1, 1, 1, 1, 1])
y.shape

(10,)

In [110]:
category_count_array = []

for target in np.unique(y):
    #array = np.where(y==target)
    #print(len(array[0]))
    print("target name", target)
    category_count_array.append(len(np.where(y==target)[0]))
    
for target in category_count_array:
    print("target", target)
print("sum",sum(category_count_array))
node_num_array = [[10, 30],[20, 5],[5, 3]]
len(node_num_array)

target name 0
target name 1
target 5
target 5
sum 10


3

In [134]:
import itertools

l = ['a', 'b', 'c', 'd']
for v in itertools.combinations(X, 2):
    print(v)

(array([1, 5]), array([2, 5]))
(array([1, 5]), array([3, 5]))
(array([1, 5]), array([1, 3]))
(array([1, 5]), array([2, 4]))
(array([1, 5]), array([3, 3]))
(array([1, 5]), array([5, 3]))
(array([1, 5]), array([4, 2]))
(array([1, 5]), array([3, 1]))
(array([1, 5]), array([5, 1]))
(array([2, 5]), array([3, 5]))
(array([2, 5]), array([1, 3]))
(array([2, 5]), array([2, 4]))
(array([2, 5]), array([3, 3]))
(array([2, 5]), array([5, 3]))
(array([2, 5]), array([4, 2]))
(array([2, 5]), array([3, 1]))
(array([2, 5]), array([5, 1]))
(array([3, 5]), array([1, 3]))
(array([3, 5]), array([2, 4]))
(array([3, 5]), array([3, 3]))
(array([3, 5]), array([5, 3]))
(array([3, 5]), array([4, 2]))
(array([3, 5]), array([3, 1]))
(array([3, 5]), array([5, 1]))
(array([1, 3]), array([2, 4]))
(array([1, 3]), array([3, 3]))
(array([1, 3]), array([5, 3]))
(array([1, 3]), array([4, 2]))
(array([1, 3]), array([3, 1]))
(array([1, 3]), array([5, 1]))
(array([2, 4]), array([3, 3]))
(array([2, 4]), array([5, 3]))
(array([

In [125]:
class ScratchDecesionTreeClassifierDepth1():
    """
    深さ1の決定木分類器のスクラッチ実装

    Parameters
    ----------
    verbose : bool
      学習過程を出力する場合はTrue
    max_depth : int
      木の深さ
      
    Atrivute
    ----------
    _node : list(int, int, int,(n_features))
      各ノードNo.、親のノードNo.、ジニ不純度、各ラベルに分類された個数
      ノード分追加
      
    _label_name : list(n_features)
      正解ラベルの名前
    """
    def __init__(self, verbose=False, max_depth=1, split_num=2):
        # ハイパーパラメータを属性として記録
        self.verbose = verbose
        self.max_depth = max_depth
        self.split_num = split_num
        
    def fit(self, X, y):
        """
        決定木分類器を学習する
        Parameters
        ----------
        X : 次の形のndarray, shape (n_samples, n_features)
            訓練データの特徴量
        y : 次の形のndarray, shape (n_samples, )
            訓練データの正解値
        """
        self._node = []
        self._label_name = []
        # rootノードの各ラベル数を正解ラベルから取得（正解ラベルに入っていないことは想定しない）
        category_count_array = []
        for target in np.unique(y):
            category_count_array.append(len(np.where(y==target)[0]))
            self._label_name.append(target)
        
        lmp = self._gini_Impure(category_count_array)
        print("不純度 : ", lmp)
        # _nodeにノード情報を保存 : 0:No. / 1:親は無いので親No.-1 / 2:ジニ不純度 /3:各ラベルの個数
        self._node.append([0, -1, lmp, category_count_array])
        print("self._node : ", self._node)
        
        
        for i in range(1, self.max_depth+1):
            print("木の深さ : ",i)
            node_num_array = [[10, 30],[20, 5]]
            parent = 0
            ig = self._Information_gain(parent, node_num_array)
            print("情報利得 : ", ig)
        
        
        if self.verbose:
            #verboseをTrueにした際は学習過程を出力
            print()
        pass
    
    def _gini_Impure(self, category_count_array):
        """
        ジニ不純度を計算
        Parameters
        ----------
        category_count_array : 次の形 list(n_features)
          各ラベルの個数
        
        return float
        """
        lmp = 1
        for target in category_count_array:
            lmp -= (target/sum(category_count_array))**2
        
        return lmp
    
    def _Information_gain(self, parent_i, node_num_array):
        """
        情報利得を計算
        Parameters
        ----------
        node_num_array : list
        """
        
        print("親のジニ不純度", self._node[parent_i][2])
        
        ig = self._node[parent_i][2]
        
        for target in node_num_array:
            print("target", target)
            lmp = self._gini_Impure(target)
            print("lmp", lmp)
            print("sum(target)", sum(target))
            print("親のサンプル数", sum(self._node[parent_i][3]))
            #ig -= lmp*sum(target)/sum(self._node[parent_i][3])
            ig -= lmp*sum(target)/65
        
        return ig
    
    def predict(self, X):
        """
        決定木分類器を使いラベルを推定する
        """
        pass
        return

In [126]:
sc_DT_1 = ScratchDecesionTreeClassifierDepth1()
sc_DT_1.fit(X, y)

不純度 :  0.5
self._node :  [[0, -1, 0.5, [5, 5]]]
木の深さ :  1
親のジニ不純度 0.5
target [10, 30]
lmp 0.375
sum(target) 40
親のサンプル数 10
target [20, 5]
lmp 0.31999999999999984
sum(target) 25
親のサンプル数 10
情報利得 :  0.1461538461538462
